In [1]:
import sys
import json
import pandas as pd
import numpy as np
import os
import boto3
from decimal import Decimal
from math import pi

ExpressionAttributeNames = {
    "#TRP": "trip",
    "#TM": "tm",
    "#RPM": "rpm",
    "#SPD": "speed",
    "#THRTL": "throttle_pos",
    "#ACC_X": "acc_x",
    "#ACC_Y": "acc_y",
    "#ACC_Z": "acc_z",
    "#GYRO_X": "gyro_x",
    "#GYRO_Y": "gyro_y",
    "#GYRO_Z": "gyro_z",
}

ProjectionExpression = "#TRP, #TM, #RPM, #SPD, #THRTL, #ACC_X, #ACC_Y, #ACC_Z, #GYRO_X, #GYRO_Y, #GYRO_Z"

# Returns a processed dataframe that contains the information needed for the heatmap
# :return: a pandas DataFrame that contains the information needed for the heatmap
def get_data(event):
    dynamodb = boto3.resource('dynamodb')
    table = dynamodb.Table('vehicle_data')
    trip_id = event["trip_id"]
        
    response = table.scan(ExpressionAttributeNames = ExpressionAttributeNames, ProjectionExpression = ProjectionExpression)
    data = pd.DataFrame.from_dict(response['Items'])

    return data[data["trip"] == trip_id]



In [2]:
def lambda_handler(event, context):
    data = get_data(event)
    avg_v = data.speed.mean()
    avg_a = np.sqrt(data.acc_x.mean() ** 2 + data.acc_y.mean() ** 2 + data.acc_z.mean() ** 2)
    trip_tm = data.tm.iloc[-1] - data.tm.iloc[0]
    avg_rpm = data.rpm.mean()
    time = json.dumps(list(data.tm.astype(float)))
    speed = json.dumps(list(data.speed.astype(float)))
    throttle_pos = json.dumps(list(data.throttle_pos.astype(float)))


    return {
        'statusCode': 200,
        'headers': {'Content-Type': 'application/json', 'Access-Control-Allow-Origin': '*', 'Access-Control-Allow-Headers': 'Access-Control-Allow-Origin'},
        'body': {"average_speed": json.dumps(avg_v), "average_acceleration": json.dumps(avg_a), 
        "average_rpm": json.dumps(float(avg_rpm)), "trip_time": json.dumps(float(trip_tm)), "time": time, "speed":speed, "throttle_position":throttle_pos
        }
    }


In [3]:
lambda_handler(event = {'trip_id':1}, context = 0)

{'statusCode': 200,
 'headers': {'Content-Type': 'application/json',
  'Access-Control-Allow-Origin': '*',
  'Access-Control-Allow-Headers': 'Access-Control-Allow-Origin'},
 'body': {'average_speed': '93.26086956521739',
  'average_acceleration': '16.209579654938373',
  'average_rpm': '5469.391304347826',
  'trip_time': '22.0',
  'time': '[1648162242.0, 1648162243.0, 1648162244.0, 1648162245.0, 1648162246.0, 1648162247.0, 1648162248.0, 1648162249.0, 1648162250.0, 1648162251.0, 1648162252.0, 1648162253.0, 1648162254.0, 1648162255.0, 1648162256.0, 1648162257.0, 1648162258.0, 1648162259.0, 1648162260.0, 1648162261.0, 1648162262.0, 1648162263.0, 1648162264.0]',
  'speed': '[129.0, 85.0, 43.0, 105.0, 148.0, 117.0, 135.0, 19.0, 71.0, 133.0, 3.0, 124.0, 70.0, 116.0, 116.0, 143.0, 95.0, 68.0, 64.0, 120.0, 42.0, 147.0, 52.0]',
  'throttle_position': '[87.0, 98.0, 10.0, 59.0, 18.0, 6.0, 67.0, 28.0, 93.0, 19.0, 62.0, 27.0, 12.0, 56.0, 21.0, 89.0, 16.0, 21.0, 74.0, 44.0, 78.0, 80.0, 67.0]'}}

In [7]:
import requests
# myHeaders = {"Content-Type", "application/json"}
# response = requests.get("https://7m3q3dqm4g.execute-api.us-east-1.amazonaws.com/dev", headers=myHeaders)
# print(response)

headers = {}
url = "https://7m3q3dqm4g.execute-api.us-east-1.amazonaws.com/dev"
method = 'POST'
redirect = 'follow'
body={"trip_id": 2}


response = requests.request(method=method, url=url, data = json.dumps(body), headers=headers)
print(json.loads(response.text))
response

{'statusCode': 200, 'headers': {'Content-Type': 'application/json', 'Access-Control-Allow-Origin': '*', 'Access-Control-Allow-Headers': 'Access-Control-Allow-Origin'}, 'body': {'average_speed': '86.375', 'average_acceleration': '29.673286070976367', 'average_rpm': '4984.75', 'trip_time': '15.0', 'time': '[1648162265.0, 1648162266.0, 1648162267.0, 1648162268.0, 1648162269.0, 1648162270.0, 1648162271.0, 1648162272.0, 1648162273.0, 1648162274.0, 1648162275.0, 1648162276.0, 1648162277.0, 1648162278.0, 1648162279.0, 1648162280.0]', 'speed': '[70.0, 34.0, 44.0, 117.0, 56.0, 27.0, 119.0, 66.0, 42.0, 128.0, 128.0, 139.0, 71.0, 48.0, 150.0, 143.0]', 'throttle_position': '[68.0, 68.0, 92.0, 67.0, 69.0, 0.0, 66.0, 16.0, 94.0, 87.0, 24.0, 21.0, 62.0, 49.0, 49.0, 66.0]'}}


<Response [200]>